# Tarea 2 - Sistema de recomendación
## Inteligencia de negocios

### Bibliotecas usadas

In [1]:
import pandas as pd
from dotenv import load_dotenv
import requests
import numpy as np
import os
from tqdm import tqdm
load_dotenv()

True

### Carga de datos y preprocesamiento

#### Lectura de los archivos

In [2]:
movie_titles = pd.read_csv("movie_titles.csv", usecols=range(3), names=["id", "year", "name"], dtype={"movie": int}, encoding="ISO-8859-1")
movie_titles

,id,year,name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [3]:

ratings_folder = 'c:\\umayor\\training_set'
files = ['\\mv_{}.txt'.format(str(f).zfill(7)) for f in range(1,17771)]
movie_ratings = pd.DataFrame(columns = ['movie', 'user', 'rating', 'date'])
df_list = []

for filename in tqdm(files):
    temp = pd.read_csv(ratings_folder + filename, names = ['user', 'rating', 'date'], skiprows = 1)
    temp['movie'] = filename.split('.')[0].split('_')[1].lstrip('0')
    df_list.append(temp)

movie_ratings = pd.concat(df_list).loc[:, ['movie', 'user', 'rating', 'date']]

100%|██████████| 17770/17770 [01:22<00:00, 214.12it/s]


In [4]:
movie_ratings

,movie,user,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03
...,...,...,...,...
916,17770,1790158,4,2005-11-01
917,17770,1608708,3,2005-07-19
918,17770,234275,1,2004-08-07
919,17770,255278,4,2004-05-28


#### Identificación y corrección de columnas vacías

In [5]:
print("----- Campos NaN en columna id -----\n", movie_titles[movie_titles['name'].isna()])
print("----- Campos NaN en columna año -----\n", movie_titles[movie_titles['year'].isna()])
print("----- Campos NaN en columna nombre -----\n", movie_titles[movie_titles['id'].isna()])

----- Campos NaN en columna id -----
 Empty DataFrame
Columns: [id, year, name]
Index: []
----- Campos NaN en columna año -----
           id  year                                         name
4387    4388   NaN      Ancient Civilizations: Rome and Pompeii
4793    4794   NaN  Ancient Civilizations: Land of the Pharaohs
7240    7241   NaN     Ancient Civilizations: Athens and Greece
10781  10782   NaN                       Roti Kapada Aur Makaan
15917  15918   NaN                      Hote Hote Pyaar Ho Gaya
16677  16678   NaN                              Jimmy Hollywood
17666  17667   NaN                           Eros Dance Dhamaka
----- Campos NaN en columna nombre -----
 Empty DataFrame
Columns: [id, year, name]
Index: []


#### Llenado de la información faltante
Para no tener que eliminar estas películas, primero probamos la API de The Movie Database, en caso de que esta pudiera entregarnos los años de cada película.

In [6]:
api_key = os.getenv("API_KEY")

for movie in movie_titles[movie_titles['year'].isna()]['name']:
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie}"
    response = requests.get(url)
    data = response.json()
    try:
        relase_date = data['results'][0]['release_date']
        if relase_date == '':
            raise
        release_year = relase_date[:4]
        print("🟢 La fecha de lanzamiento de " + movie + " es " + relase_date + ", del año " + release_year + ".")
        movie_titles.loc[movie_titles['name'] == movie, 'year'] = release_year
    except: print("🔴 No se encontró fecha de lanzamiento para " + movie + ".")

🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Rome and Pompeii.
🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Land of the Pharaohs.
🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Athens and Greece.
🟢 La fecha de lanzamiento de Roti Kapada Aur Makaan es 1974-01-01, del año 1974.
🔴 No se encontró fecha de lanzamiento para Hote Hote Pyaar Ho Gaya.
🟢 La fecha de lanzamiento de Jimmy Hollywood es 1994-03-30, del año 1994.
🔴 No se encontró fecha de lanzamiento para Eros Dance Dhamaka.


Ya que solo se encontró el año de lanzamiento de dos películas, se procede a hacer una búsqueda manual en Google para llenar las películas faltantes.

In [7]:
movie_titles.at[4387, 'year'] = 2001
movie_titles.at[4793, 'year'] = 2001
movie_titles.at[7240, 'year'] = 2002
movie_titles.at[15917, 'year'] = 1999
movie_titles.at[17666, 'year'] = 1999
print("----- Campos NaN en columna año -----\n", movie_titles[movie_titles['year'].isna()])

----- Campos NaN en columna año -----
 Empty DataFrame
Columns: [id, year, name]
Index: []
